In [1]:
# from google.colab import drive
# drive.mount('/content/drive/')

In [2]:
# %cd drive/MyDrive/skripsi/

In [3]:
# !pip uninstall -y tensorflow 
# !pip install tensorflow==2.0

# !pip uninstall -y h5py 
# !pip install h5py==2.10

In [4]:
import tensorflow as tf, matplotlib.pyplot as plt, os, numpy as np, matplotlib.pyplot as plt, matplotlib.patches as patches
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from framework.utils import bbox_utils, data_utils, drawing_utils, eval_utils, io_utils, train_utils
from framework.models import faster_rcnn

tf.__version__  # if not 2.0.0 please uncomment pip uninstall and install, make sure your python version is 3.7.x

'2.1.0'

In [5]:
import h5py
h5py.__version__  # if not 2.10.0 please uncomment pip uninstall and install, make sure your python version is 3.7.x

'2.10.0'

# Read Dataset from TFRecord

In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
IMAGE_SIZE = 512  # make sure had same size with the picture

In [ ]:
def read_tfrecord(example):
    tfrecord_format = {
        "filename": tf.io.FixedLenFeature([], tf.string),
        "pic": tf.io.FixedLenFeature([], tf.string),
        "bbox": tf.io.FixedLenFeature([], tf.string),
        "label": tf.io.FixedLenFeature([], tf.string)
    }
    example = tf.io.parse_single_example(example, tfrecord_format)
    filename = tf.cast(example["filename"], tf.string)
    image = tf.io.parse_tensor(example["pic"], out_type = tf.uint8)
    bbox = tf.io.parse_tensor(example["bbox"], out_type = tf.float32)
    label = tf.io.parse_tensor(example["label"], out_type = tf.int32)
    return {"filename": filename, "image": image, "bbox": bbox, "label": label}

In [ ]:
def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False  # disable order, increase speed
    dataset = tf.data.TFRecordDataset(filenames)  # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order)  # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_tfrecord)
    return dataset

In [ ]:
def get_dataset(filenames):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    return dataset

In [ ]:
def read_label_map(label_map_path):
    item_id = None
    item_name = None
    items = {}

    with open(label_map_path, "r") as file:
        for line in file:
            line.replace(" ", "")
            if line == "item{":
                pass
            elif line == "}":
                pass
            elif "id" in line:
                item_id = int(line.split(":", 1)[1].strip())
            elif "name" in line:
                item_name = line.split(":", 1)[1].replace("'", "").replace("\"", "").strip()

            if item_id is not None and item_name is not None:
                items[item_name] = item_id
                item_id = None
                item_name = None

    return items

In [ ]:
label_map_path = "./data_preparation/label_map.pbtxt"
label_map_dict = read_label_map(label_map_path)

In [ ]:
def get_label_text(result, doc = label_map_dict):
    for key, value in doc.items():
        if(value == result + 1):
            return key
    return "Unpredictable"

In [ ]:
train_data = get_dataset("./data_preparation/train.tfrecord")

test_data = get_dataset("./data_preparation/test.tfrecord")

train_data

In [ ]:
def show_data(data, n):
    print(data)
    for dat in data.take(n):
        plt.imshow(dat["image"])
        for coord in dat["bbox"]: # bbox is ymin, xmin, ymax, xmax
            coord *= IMAGE_SIZE
            rect = patches.Rectangle(
                (coord[1].numpy(), coord[0].numpy()),  # x1, y1
                coord[3].numpy() - coord[1].numpy(),  # width
                coord[2].numpy() - coord[0].numpy(),  # height
                linewidth = 2, edgecolor = "r", fill = False)
            plt.gca().add_patch(rect)
        plt.show()

In [ ]:
# show_data(train_data, 20)

# Train Model

In [ ]:
batch_size = 4
epochs = 10
load_weights = False
backbone = "resnet50"

hyper_params = train_utils.get_hyper_params(backbone)
train_total_item = len(list(train_data))

In [ ]:
labels = list(label_map_dict.keys())
# We add 1 class for background
hyper_params["total_labels"] = len(labels) + 1
train_data = train_data.map(lambda data : data_utils.preprocessing_before_frcnn(data, IMAGE_SIZE, IMAGE_SIZE))

In [ ]:
data_shapes = data_utils.get_data_shapes()
padding_values = data_utils.get_padding_values()
train_data = train_data.padded_batch(batch_size, padded_shapes=data_shapes, padding_values=padding_values)

In [ ]:
anchors = bbox_utils.generate_anchors(hyper_params)
frcnn_train_feed = train_utils.faster_rcnn_generator(train_data, anchors, hyper_params)

In [ ]:
if (backbone == "vgg16"):
    from framework.models.rpn_vgg16 import get_rpn_model
elif (backbone == "mobilenet_v2"):
    from framework.models.rpn_mobilenet_v2 import get_rpn_model
elif (backbone == "resnet50"):
    from framework.models.rpn_resnet50 import get_rpn_model

rpn_model, feature_extractor = get_rpn_model(hyper_params)
frcnn_model = faster_rcnn.get_model_frcnn(feature_extractor, rpn_model, anchors, hyper_params)
frcnn_model.compile(optimizer=tf.optimizers.SGD(learning_rate=18e-4),
                    loss=[None] * len(frcnn_model.output))
faster_rcnn.init_model_frcnn(frcnn_model, hyper_params)

In [ ]:
# Load weights
frcnn_model_path = io_utils.get_model_path("faster_rcnn", backbone)

load_weights = False
if load_weights:
    frcnn_model.load_weights(frcnn_model_path)
log_path = io_utils.get_log_path("faster_rcnn", backbone)

checkpoint_callback = ModelCheckpoint(frcnn_model_path, monitor="loss", save_best_only=True, save_weights_only=True)
# tensorboard_callback = TensorBoard(log_dir=log_path)

In [ ]:
step_size_train = train_utils.get_step_size(train_total_item, batch_size)
history = frcnn_model.fit(frcnn_train_feed,
                steps_per_epoch=step_size_train,
                verbose = 1,
                epochs=epochs,
                callbacks=[checkpoint_callback])

# Evaluate Model

In [ ]:
# labels = ["bg"] + labels
# test_total_item = len(list(test_data))

# test_data = test_data.map(lambda data : data_utils.preprocessing_before_frcnn(
#                               data, IMAGE_SIZE, IMAGE_SIZE))
# test_data = test_data.padded_batch(
#     batch_size, padded_shapes=data_shapes, padding_values=padding_values)

In [ ]:
# load_path = io_utils.get_model_path("faster_rcnn", backbone)
# rpn_model, feature_extractor = rpn_vgg16.get_model_vgg16(hyper_params)
# frcnn_test_model = faster_rcnn.get_model_frcnn(feature_extractor, rpn_model, anchors, hyper_params, mode="test")
# frcnn_test_model.load_weights(load_path)

In [ ]:
# hyper_params["total_labels"]

In [ ]:
# step_size = train_utils.get_step_size(test_total_item, batch_size)
# pred_bboxes, pred_labels, pred_scores = frcnn_test_model.predict(test_data, steps=step_size, verbose=1)

In [ ]:
# pred_bboxes

In [ ]:
# pred_labels

In [ ]:
# pred_scores

In [ ]:
# eval_utils.evaluate_predictions(test_data, pred_bboxes, pred_labels, pred_scores, labels, batch_size)